In [24]:
import pandas as pd
import numpy as np
import random
import math

In [25]:
# COUNT Completion Time of sequence of order in flowshop schedule
def count_completion_time(processing_times):
    process_accumlutaion = np.zeros(processing_times.shape)
    accumulation_time = 0
    for i in range(len(processing_times[0])):
        accumulation_time += processing_times[0][i]
        process_accumlutaion[0][i] = accumulation_time

    for i in range(1, len(processing_times)):
        accumulation_time = process_accumlutaion[i-1][0]
        for j in range(len(processing_times[0])):
            if(j == 0):
                accumulation_time += processing_times[i][j]
            else:
                if(process_accumlutaion[i-1][j] < process_accumlutaion[i][j-1]):
                    accumulation_time += processing_times[i][j]
                else:
                    accumulation_time = process_accumlutaion[i-1][j]
                    accumulation_time += processing_times[i][j]
            process_accumlutaion[i][j] = accumulation_time
    return accumulation_time

In [26]:
def generate_random_sequence():
  """Generates a random sequence of numbers from 1 to 6."""
  sequence = random.sample(range(1, 7), 6)  # Sample 6 unique numbers from range 1 to 6
  return ''.join(map(str, sequence))  # Convert sequence to string with '-' separator


In [27]:
# Generate Random Solution of amount_of_solution in return of dictionary with data
# {"sequence" : "compleiton_time"}
# ...
def generate_random_solution(data, amount_of_solution:int):
    sol = dict()
    for i in range(amount_of_solution):
        seq = generate_random_sequence()
        seq_list = np.zeros(data.shape)
        for i in range(len(seq)):
            seq_list[i] = data[int(seq[i])-1]
        sol.update({seq: count_completion_time(seq_list)})
    return sol


In [28]:
def min_max_normalization(arr):
    min_val = np.min(arr) -1 # (np.min(arr)  * 1/100)
    max_val = np.max(arr)
    return (arr - min_val) / (max_val - min_val)

In [29]:
# Z-Score Normalization 
def zscore_normalization(arr):
    mean_val = np.mean(arr)
    std_dev = np.std(arr)
    return (arr - mean_val) / std_dev


In [30]:
def find_up_low_bound(arr):
    max_val = np.max(arr)
    min_val = np.min(arr)
    up_bound = math.ceil(max_val)
    low_bound = math.floor(min_val)
    return low_bound, up_bound

In [31]:
def sort_by_index(data):
  """Sorts a list and returns the corresponding sorted index sequence.

  Args:
      data: A list of sortable elements.

  Returns:
      A list of indices representing the sorted order of the original list.
  """
  return sorted(range(len(data)), key=lambda i: data[i])


### RUN

In [32]:
#LOAD DATA
data = pd.read_csv("data.csv")
data = data.values
data = data[:, 1:]


In [33]:
# INIT THE PARAMETERS
population_size = 3
iteration_ = 3
gamma = 0.01
alpha = 0.1
beta = 0.2
delta = 0.97

In [34]:
# Generate Random Solution based on population_size
solution_ = generate_random_solution(data=data, amount_of_solution=population_size)

In [35]:
solution_keys = list(solution_.keys())
solution_list = list(solution_.values())
solution_list = np.array(solution_list)

In [36]:
normalize_solution = zscore_normalization(solution_list)

In [37]:
low_bound, up_bound  = find_up_low_bound(normalize_solution)

In [38]:
# CREATE OBJECT AND RUN ALGORITHM
from firefly import firefly_algo

In [39]:
fa = firefly_algo(data=normalize_solution, population=population_size, max_iter=iteration_, low_bound=low_bound, up_bound=up_bound, gamma=gamma, alpha=alpha, beta=beta, delta=delta)
fa.run()

In [40]:
fa.print_result()

1.069
-4.760
0.267


In [43]:
sorted_indices = sort_by_index(normalize_solution)

In [45]:
res = dict()
for i in sorted_indices:
    res.update({solution_keys[i]: fa.data[i]})

In [46]:
res

{'342156': -4.759963380238933,
 '546321': 0.2672612419124206,
 '614325': 1.0690449676496938}